## Création d'un Mapper par application de la transformée de Burrow-Wheeler

### 1) Génération des permutations et tri

A partir d'une séquence (à laquelle nous avons ajouté le symbole dollar), nous allons générer toutes les permutations possibles en décalant le dollar (plus petit caractère) de une position vers la droite.

In [143]:
def permutator(seq):
    permutations = []
    permutations.append(list(seq))
    for i in range(1, len(seq)):
        seq = seq[-1] + seq[:-1]
        permutations.append(list(seq))
    return permutations

tri = lambda permut: sorted(permut)

In [137]:
seq = "$CAATGCAA"
permutations = permutator(seq)

Les permutations doivent ensuites êtres triées par ordre alphabétique.

In [140]:
permutations = tri(permutations)
permutations

[['$', 'C', 'A', 'A', 'T', 'G', 'C', 'A', 'A'],
 ['A', '$', 'C', 'A', 'A', 'T', 'G', 'C', 'A'],
 ['A', 'A', '$', 'C', 'A', 'A', 'T', 'G', 'C'],
 ['A', 'A', 'T', 'G', 'C', 'A', 'A', '$', 'C'],
 ['A', 'T', 'G', 'C', 'A', 'A', '$', 'C', 'A'],
 ['C', 'A', 'A', '$', 'C', 'A', 'A', 'T', 'G'],
 ['C', 'A', 'A', 'T', 'G', 'C', 'A', 'A', '$'],
 ['G', 'C', 'A', 'A', '$', 'C', 'A', 'A', 'T'],
 ['T', 'G', 'C', 'A', 'A', '$', 'C', 'A', 'A']]

### 2) Récupération des caractéristiques

Trois caractéristiques nous intéressent: la première colonne, la dernière colonne, et la position du dollar dans chacune des permutations.

In [148]:
def dollar_pos(permutation_matrix):
    pos = []
    for permutation in permutation_matrix:
        pos.append(permutation.index("$"))
    return pos

def first_col(permutation_matrix):
    col = []
    for permutation in permutation_matrix:
        col.append(permutation[0])
    return col

def last_col(permutation_matrix):
    col = []
    for permutation in permutation_matrix:
        col.append(permutation[-1])
    return col

In [145]:
dollar = dollar_pos(permutations)
print(dollar)
first = first_col(permutations)
print(first)
last = last_col(permutations)
print(last)

[0, 1, 2, 7, 6, 3, 8, 4, 5]
['$', 'A', 'A', 'A', 'A', 'C', 'C', 'G', 'T']
['A', 'A', 'C', 'C', 'A', 'G', '$', 'T', 'A']


### 3) Corps de l'algorithme

Dans un premier temps nous voulons générer une liste de doublet où le premier élément est un caractère de la première colonne, et le second est la position où l'on retrouve ce caractère dans la dernière colonne.

In [149]:
from copy import deepcopy

def doublet_generator(first_col, last_col):
    all_doublet = []
    last_col_copy = deepcopy(last_col)
    for i in range(len(first_col)):
        doublet = []
        doublet.append(first_col[i])
        doublet.append(last_col_copy.index(first_col[i]))
        last_col_copy[last_col_copy.index(first_col[i])] = None
        all_doublet.append(doublet)
    return all_doublet

In [150]:
all_doublet = doublet_generator(first, last)

In [151]:
all_doublet

[['$', 6],
 ['A', 0],
 ['A', 1],
 ['A', 4],
 ['A', 8],
 ['C', 2],
 ['C', 3],
 ['G', 5],
 ['T', 7]]

On veut ensuite trier les doublet en fonction de leur second élement (numérique donc). Ainsi, on retombe sur la séquence de départ. C'est à ca que nous as servit la première colonne. d'ordinaire on ne garde que la dernière.

In [156]:
def reversion(liste):   
    seq = []
    flag = 0
    for i in range(len(liste)):
        seq.append(liste[flag][0])
        flag = liste[flag][1]
    return "".join(seq)

In [157]:
revert_seq = reversion(all_doublet)
print(revert_seq)

$CAATGCAA


Nous allons construire un index qui répértorie combien il y a de "A" avant le dollar de la séquence courante (dernière colonne), et faire de même pour les autres types de bases.

In [166]:
def index_generator(liste):
    index = []
    for i in range(len(liste)+1):
        if i ==0:
            d = {"$":0,"A":0,"C":0,"G":0,"T":0}
        else:
            d = {"$":liste[0:i].count("$"),
                 "A":liste[0:i].count("A"),
                 "C":liste[0:i].count("C"),
                 "G":liste[0:i].count("G"),
                 "T":liste[0:i].count("T")}
        index.append(d)
    return index

In [170]:
index = index_generator(last)
index

[{'$': 0, 'A': 0, 'C': 0, 'G': 0, 'T': 0},
 {'$': 0, 'A': 1, 'C': 0, 'G': 0, 'T': 0},
 {'$': 0, 'A': 2, 'C': 0, 'G': 0, 'T': 0},
 {'$': 0, 'A': 2, 'C': 1, 'G': 0, 'T': 0},
 {'$': 0, 'A': 2, 'C': 2, 'G': 0, 'T': 0},
 {'$': 0, 'A': 3, 'C': 2, 'G': 0, 'T': 0},
 {'$': 0, 'A': 3, 'C': 2, 'G': 1, 'T': 0},
 {'$': 1, 'A': 3, 'C': 2, 'G': 1, 'T': 0},
 {'$': 1, 'A': 3, 'C': 2, 'G': 1, 'T': 1},
 {'$': 1, 'A': 4, 'C': 2, 'G': 1, 'T': 1}]

Construction d'un surindex, je ne sais pas trop pourquoi...

In [201]:
def surindex_generator(index):
    surindex = {"$":0,"A":0,"C":0,"G":0,"T":0}
    keys = list(surindex.keys())
    for i in range(1, len(keys)):
        surindex[keys[i]] = surindex[keys[i-1]] + index[keys[i-1]]
    return surindex

In [202]:
surindex = surindex_generator(index[-1])
surindex

{'$': 0, 'A': 1, 'C': 5, 'G': 7, 'T': 8}

Fonction finale;

In [204]:
def final(read, surindex, index, last_col):
    p = len(read)-1
    carac = read[p]
    b =  surindex[carac]
    e = b + index[len(last_col)][carac]
    end = []
    while (p >0 and e > b):
        p -= 1
        carac = read[p]
        b =  surindex[carac] + index[b][carac]
        e = surindex[carac] + index[e][carac]
    end.append(b)
    end.append(e)
    return end

In [207]:
fin = final("CAA",surindex, index, last)
print(fin)

[5, 7]


#### <u> Nombre de hits </u>:

In [208]:
print(fin[1] - fin[0])

2


#### <u>Régénération de la séquence initiale <u/>:

In [215]:
appariement = list(zip(dollar, last))
appariement = sorted(appariement, reverse = True)
appariement

[(8, '$'),
 (7, 'C'),
 (6, 'A'),
 (5, 'A'),
 (4, 'T'),
 (3, 'G'),
 (2, 'C'),
 (1, 'A'),
 (0, 'A')]

In [217]:
seq_initiale = []
for tuples in appariement:
    caract = tuples[1]
    seq_initiale.append(caract)
seq_initiale.remove("$")
seq_initiale = "".join(seq_initiale)

print(seq_initiale)

CAATGCAA
